<img src="https://drive.google.com/uc?export=view&id=14reVO1X6LsjqJ3cFgoeHxxddZVGfZn3t" width="100%">

In [1]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os
import sys
import kagglehub

from sklearn.model_selection import train_test_split
import keras
import keras_tuner
import gdown
import zipfile

In [3]:
path = kagglehub.dataset_download("vencerlanz09/agricultural-pests-image-dataset")

In [4]:
scripts_url = "https://drive.google.com/uc?id=1Ua3O6uh45uNOfcIbPZch2uDHeGUVmpL8"
zip_path = "/content/scripts.zip"
gdown.download(scripts_url, zip_path)

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall()
    print("Extracción completada.")

else:
    print(f"Archivo no encontrado: {zip_path}.")

script_folder = "/content/scripts"

Downloading...
From (original): https://drive.google.com/uc?id=1Ua3O6uh45uNOfcIbPZch2uDHeGUVmpL8
From (redirected): https://drive.google.com/uc?id=1Ua3O6uh45uNOfcIbPZch2uDHeGUVmpL8&confirm=t&uuid=6c9ff8bc-5190-4fa5-892f-3a52d14eb72b
To: /content/scripts.zip
100%|██████████| 377M/377M [00:05<00:00, 66.6MB/s]


Extracción completada.


In [5]:
image_path = []
image_class = []
labels = []

classes = os.listdir(path)
for label, Class in enumerate(classes):
    images_names = os.listdir(f'{path}/{Class}')
    image_path = image_path +  [f'{path}/{Class}/{name}' for name in images_names]
    image_class = image_class + len(images_names)*[Class]
    labels = labels + len(images_names)*[label]

metadata = pd.DataFrame(np.array([image_path, image_class, labels]).T, columns=['path','class name','label'])
metadata.head()

,path,class name,label
0,/kaggle/input/agricultural-pests-image-dataset...,beetle,0
1,/kaggle/input/agricultural-pests-image-dataset...,beetle,0
2,/kaggle/input/agricultural-pests-image-dataset...,beetle,0
3,/kaggle/input/agricultural-pests-image-dataset...,beetle,0
4,/kaggle/input/agricultural-pests-image-dataset...,beetle,0


In [6]:
metadata_train, metadata_test = train_test_split(metadata, test_size=0.15, random_state=42)
metadata_train, metadata_tune = train_test_split(metadata_train, test_size=0.175, random_state=42)

print(f'Numero de registros de entrenamiento: {metadata_train.shape[0]}')
print(f'Numero de registros de validación: {metadata_tune.shape[0]}')
print(f'Numero de registros de Ajuste: {metadata_test.shape[0]}')

Numero de registros de entrenamiento: 3851
Numero de registros de validación: 818
Numero de registros de Ajuste: 825


In [7]:
from scripts.preprocessing import DataGenerator as gd

In [8]:
train_generator = gd.DataGenerator(metadata_train, batch_size=32, dim=(128,128,3), shuffle=True)
test_generator = gd.DataGenerator(metadata_test, batch_size=32, dim=(128,128,3), shuffle=True)
tune_generator = gd.DataGenerator(metadata_tune, batch_size=32, dim=(128,128,3), shuffle=True)

X, Y = train_generator.__getitem__(0)
print(f'Dimensión de tensor de entrada: {X.shape}')
print(f'Dimensión de tensor de salida: {Y.shape}')

Dimensión de tensor de entrada: (32, 128, 128, 3)
Dimensión de tensor de salida: (32, 1)


In [9]:
def build_model(hp):
    keras.backend.clear_session()

    bb = keras.applications.VGG19(include_top=False, weights="imagenet", input_shape=(128,128,3))
    for layer in bb.layers:
        layer.trainable=False

    model = keras.Sequential()
    model.add(keras.layers.Input((128,128,3)))
    model.add(bb)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(hp.Choice('units', [64, 128, 256]), activation='relu'))
    model.add(keras.layers.Dense(12, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return  model

In [10]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=3,
    executions_per_trial=3,
    overwrite=False,
    directory=f"{script_folder}/training/",
    project_name="plagas",
)

Reloading Tuner from /content/scripts/training/plagas/tuner0.json


In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
model = build_model(best_hps[0])

checkpoint_filepath = f'{script_folder}/evaluation/best_model_weights.keras'
model_filepath = f'{script_folder}/evaluation/model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', save_best_only=True, mode='max', verbose=0)

model.fit(train_generator, epochs=15, validation_data=(tune_generator), callbacks=[model_checkpoint_callback])
model.save(model_filepath)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 1344s 11s/step - accuracy: 0.2556 - loss: 2.2576 - val_accuracy: 0.4525 - val_loss: 1.7695
Epoch 2/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 1307s 11s/step - accuracy: 0.4812 - loss: 1.6285 - val_accuracy: 0.4825 - val_loss: 1.5925
Epoch 3/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 1290s 11s/step - accuracy: 0.5423 - loss: 1.4351 - val_accuracy: 0.4975 - val_loss: 1.5171
Epoch 4/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 1334s 11s/step - accuracy: 0.5735 - loss: 1.3018 - val_accuracy: 0.5275 - val_loss: 1.4024
Epoch 5/15
 13/120 ━━━━━━━━━━━━━━━━━━━━ 15:56 9s/step - accuracy: 0.6226 - loss: 1.1793

* **Profesores:**
  - [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)
* **Asistentes docentes:**
    - [Juan Sebastián Malagón Torres](https://co.linkedin.com/in/juan-sebastian-malag%C3%B3n-torres-86039a164).
* **Diseño de imágenes:**
    - [Sebastián Daniel Moreno Martinez](http://www.linkedin.com/in/sm-xwx).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    